### 메모리 관리 방법

- 참조 카운트

: 모든 객체는 참조 카운트가 0인 상태로 생성되며, 다른 객체 참조시 1씩 증가하는 방법

- 순환 참조

: 참조 카운트로는 해결할 수 없는 문제를 의미 / 예를 들어 대상이 서로 물려있어 참조를 할 수 없을 때 사용

### 약한 참조

- 객체를 참조는 하되 카운트가 증가하진 않음

In [1]:
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)
b

<weakref at 0x7cb0402d5990; to 'numpy.ndarray' at 0x7cb02fd5f930>

In [2]:
b()

array([1, 2, 3])

In [5]:
a = None
b = weakref.ref(a) #a = None으로 두면서 작동되지않는 것으로보아 인스턴스 삭제됨
b()

TypeError: cannot create weak reference to 'NoneType' object

### weakref 구조에 dezero 도입

: 구조 이해용도

In [9]:
self.inputs = inputs

self.outputs = [weakref(output) for output in outpus]
return outputs if len(outputs) > 1 else outputs[0]

NameError: name 'inputs' is not defined

In [10]:
funcs = [self.creator]
while funcs:
  f = funcs.pop()
  gys = [output().grad for output in f.outputs]
  gxs = f.backward(*gys)

NameError: name 'self' is not defined

### 변수 사용성 개선 방법

: variable 더욱 쉽게 사용하기 위해 개선

In [11]:
class Variable:
  def __init__(self, data, name = None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.name = name
    self.grad = None
    self.creator = None
    self.generation = []


In [12]:
class Variable:
  @property
  def ndin(self):
    return self.data.ndin

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

#ndin, size, dtype 등을 변수로 사용가능하게 됨

In [15]:
import numpy as np

class Variable:
  ''''''
  def __init__(self, data):
        self.data = data

  def __len__(self):
    return len(self.data)

x = Variable(np.array([[1,2,3], [4,5,6]]))
print(len(x))

2


In [17]:
class Variable:
  ''''''
  def __init__(self, data):
        self.data = data

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n', '\n' + ' '*9)
    return 'variable(' + p + ')'

x = Variable(np.array([[1,2,3], [4,5,6]]))
print(x)

variable([[1 2 3]
          [4 5 6]])


### 연산자 오버로드

: 연산자를 지원할 수 있도록 variable를 확장함(덧셈 & 곱셈 & 연산자 등)

: variable -> ndarray 사용하도록 구성함(y = a * b)

In [23]:
class Mul():
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.input[0].data, self.inputs[1].data
    return gy * x1, gy * x0

def mul(x0, x1):
  return Mul()(x0, x1)

In [25]:
def add(x0, x1):
  return Add()(x0, x1)

In [26]:
#곱셈 연산자

class Variable:
  def __mul__(self, other):
    return mul(self, other)

class Variable:
  Variable.__mul__ = mul
  Variable.__add__ = add

In [27]:
# variable + ndarray 함께 사용방법

def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)


In [28]:
# variable -> float & int useing

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

문제점
1. 첫 번째 인수가 float & int 경우

: __mul__메서드 호출을 시도

float & int 타입으로 인해 메서드 구현 x

- 해결방법 : 이항 연산자는 피연산자의 위치에 따라 호출되는 특수 메서드가 다름
따라서 rmul을 이용하여 해결

ex) Variable.__rmul__ = mul

2. 좌항이 ndarray 인스턴인 경우

: 연산자의 우선순위를 지정해줘야함

ex) neg / sub / rsub / truediv / pow 등

In [29]:
#Neg 구현

class Neg(function):
  def forward(self, x):
    return -x

  def backward(self, gy):
    return -gy

def neg(x):
  return Neg()(x)

Variable.__neg__ = neg

NameError: name 'function' is not defined

In [30]:
#뺄셈 미분

class Sub(Function):
  def forward(self, x0, x1):
    y = x0 - x1
    return y

  def backward(self, gy):
    return gy, -gy

def sub(x0, x1):
  x1 = as_array(x1)
  return Sub()(x0, x1)

Variable.__sub__ = sub

NameError: name 'Function' is not defined

In [31]:
#뺄셈 미분에서 발생하는 오류를 잡기위한 코드
#rsub을 정의한 후 인수의 순서를 바꿈

def rsub(x0, x1):
  x1 = as_array(x1)
  return sub(x1, x0)

Variable.__rsub__ = rsub

In [32]:
#거듭제곱의 미분

class Pow(Function):
  def __init__(self, c):
    self.c = c

  def forward(self, x):
    y = x ** self.c
    return y

  def backward(self, gy):
    x = self.input[0].data
    c = self.c

    gx  = c * x ** (c - 1) * gy
    return gx

def pow(x, c):
  return  Pow(c)(x)

Variable.__pow__ = pow

NameError: name 'Function' is not defined

### 패키지로 구성

- dezero : 딥러닝프레임워크

- steps : 01 ~ 60로 구성

In [38]:
!pip install dezero

ERROR: Could not find a version that satisfies the requirement dezero.core (from versions: none)
ERROR: No matching distribution found for dezero.core


In [39]:
import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
print(x)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations